In [1]:
#Change directory to data location
import os
path = 'E:\Daily Reanalisis ERA 5 v2'
os.chdir(path)

In [2]:
import warnings
warnings.simplefilter('ignore') #ignores simple warning

In [3]:
from datetime import datetime
import numpy as np
import pandas as pd
import xarray as xr
from tqdm import tqdm

In [4]:
#Read time-event excel data
censevt=pd.read_excel('calculation_output\CENS Event 3 days Running Mean.xlsx')
censevt=censevt.set_index('time_cens')

In [5]:
#Date event CENS CS no CS (1-->CS, 0-->no CS)
n_cens = 2
evt_cens_cs = []
for i in range(n_cens):
    evt_cens = censevt[censevt['cs event'] == i].index
    print("Banyak kejadian CENS {i} = {n} dari {x}".format(i=i, n=len(evt_cens), x=len(censevt)))
    print("Banyak kejadian CENS {i} = {n}%".format(i=i, n=(len(evt_cens)*100/len(censevt.index))))
    evt_cens_cs.append(evt_cens.values)

Banyak kejadian CENS 0 = 48 dari 117
Banyak kejadian CENS 0 = 41.02564102564103%
Banyak kejadian CENS 1 = 69 dari 117
Banyak kejadian CENS 1 = 58.97435897435897%


In [9]:
#Read MJO excel
mjoevt=pd.read_excel('data\MJO BOM.xlsx')
mjoslice=mjoevt.set_index(['Tanggal']).loc[evt_cens_cs[0]]
tglcensmjo=pd.DataFrame({'time_censnocs': evt_cens_cs[0],
                         'phase mjo':mjoslice['phase'].values,
                         'amp mjo'  :mjoslice['amplitude'].values
                          })
tglcensmjoph=tglcensmjo.set_index(['amp mjo', 'phase mjo'])
tglcensmjoamp1=tglcensmjoph[tglcensmjoph.index.get_level_values('amp mjo')>=0.5]
tglcensmjoph7=tglcensmjoamp1[tglcensmjoamp1.index.get_level_values('phase mjo')==7]
tglcensmjoph6=tglcensmjoamp1[tglcensmjoamp1.index.get_level_values('phase mjo')==6]
tglcensmjoph67=np.concatenate((tglcensmjoph6['time_censnocs'].values,tglcensmjoph7['time_censnocs'].values))
tglcensmjoph4=tglcensmjoamp1[tglcensmjoamp1.index.get_level_values('phase mjo')==4]
tglcensmjoph5=tglcensmjoamp1[tglcensmjoamp1.index.get_level_values('phase mjo')==5]
tglcensmjoph45=np.concatenate((tglcensmjoph4['time_censnocs'].values,tglcensmjoph5['time_censnocs'].values))
tglmjo4567=np.concatenate((tglcensmjoph45, tglcensmjoph67))

In [10]:
#Read EW MRG excel
ewevt=pd.read_excel('calculation_output\Indeks ER MRG Double Side ER EWNS.xlsx')
ewslice=ewevt.set_index(['time']).loc[evt_cens_cs[0]-np.timedelta64(0,'D')]

#Klimatologi Amplitude EW
ewevtxarray = xr.Dataset(
     {"amp mrg": (("time"), ewevt['amp mrg'].values), "amp er": (("time"), ewevt['amp er'].values)},
    coords={
    "time":   ewevt['time'].values},
    )
ewevtdjf=ewevtxarray.sel(time=slice('1979-12-01T00:00:00.000000000','2020-02-29T00:00:00.000000000'))
ewevtdjf=ewevtdjf.sel(time=ewevtdjf.time.dt.season=="DJF")

thresmrg=(ewevtdjf['amp mrg'].mean()+0*ewevtdjf['amp mrg'].std()).values
print('Thres Amplitude MRG: '+ str(thresmrg))
tglcensew=pd.DataFrame({'time_censnocs': evt_cens_cs[0],
                         'phase mrg':ewslice['phase mrg'].values,
                         'amp mrg'  :ewslice['amp mrg'].values,
                          })
tglcensew=tglcensew.set_index(['phase mrg', 'amp mrg'])
tglcensmrgamp = tglcensew[tglcensew.index.get_level_values('amp mrg')>=thresmrg]
tglcensmrgph1=tglcensmrgamp[tglcensmrgamp.index.get_level_values('phase mrg')==1]
tglcensmrg1=tglcensmrgph1['time_censnocs'].values

threser=(ewevtdjf['amp er'].mean()+0*ewevtdjf['amp er'].std()).values
print('Thres Amplitude ER: '+ str(threser))
tglcensew=pd.DataFrame({'time_censnocs': evt_cens_cs[0],
                         'phase er':ewslice['phase er'].values,
                         'amp er'  :ewslice['amp er'].values,
                          })
tglcensew=tglcensew.set_index(['phase er', 'amp er'])
tglcenseramp = tglcensew[tglcensew.index.get_level_values('amp er')>=threser]
tglcenserph1=tglcenseramp[tglcenseramp.index.get_level_values('phase er')==1]
tglcenser1=tglcenserph1['time_censnocs'].values

Thres Amplitude MRG: 1.2258041202083383
Thres Amplitude ER: 1.3604651167535824


In [13]:
#Read TC excel data
tcevt=pd.read_excel('data/Data Tropical Cyclone 95_125 _25.xlsx')
tcevt['DATE']= pd.to_datetime(tcevt['DATE'], format='%Y-%m-%d')
fcensevt=censevt[censevt['cs event']==0]
fcensevt['TC']=np.isin(fcensevt.index,tcevt['DATE'])*1
tcevtfilt=tcevt.set_index(['DATE'])
tcevtfilt=tcevtfilt.groupby(tcevtfilt.index.floor('D')).mean(numeric_only=True)
tgltc=fcensevt[(fcensevt['TC']==1)].index.values

In [15]:
evtmjo4567=np.empty([len(evt_cens_cs[0])])
evtmrg1=np.empty([len(evt_cens_cs[0])])
evter1=np.empty([len(evt_cens_cs[0])])
evttc=np.empty([len(evt_cens_cs[0])])
for i in range(len(evt_cens_cs[0])):
    if evt_cens_cs[0][i] in tglmjo4567:
        evtmjo4567[i]=1
    else:
        evtmjo4567[i]=0
    if evt_cens_cs[0][i] in tglcensmrg1:
        evtmrg1[i]=1
    else:
        evtmrg1[i]=0
    if evt_cens_cs[0][i] in tglcenser1:
        evter1[i]=1
    else:
        evter1[i]=0
    if evt_cens_cs[0][i] in tgltc:
        evttc[i]=1
    else:
        evttc[i]=0

In [17]:
fcensevent=pd.DataFrame({'time_censnocs': evt_cens_cs[0],
                         'MJO4567':evtmjo4567,
                         'MRG1'  :evtmrg1,
                         'ER1'  :evter1,
                         'TC'  :evttc
                          })

In [19]:
#Save to excel file
writer = pd.ExcelWriter('calculation_output\Phenomena Associated with CENS-noCS.xlsx')
fcensevent.to_excel(writer, 'Phenomena')
writer.close()